## **Limpieza del CSV de "Emergencias UNGRD"**
----

#### ***Importación de paquetes***

In [19]:
import pandas as pd
import requests

#### ***Cargar el dataset mediante la API de Datos Abiertos***

In [20]:
# Definir la URL base de la API
base_url = "https://www.datos.gov.co/resource/wwkg-r6te.json"

# Inicializar el offset, que es la posición de la fila a partir de la cual se obtendrán los datos
offset = 0

emergencias = []

# Bucle para obtener todas las filas, paginando según sea necesario
while True:
    params = {
        "$limit": 1000,
        "$offset": offset
    }

    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        break
    
    data = response.json()
    
    if not data:
        break
    
    emergencias.extend(data)
    
    # Incrementar el offset para obtener la siguiente página
    offset += 1000

print(f"Total de filas obtenidas: {len(emergencias)}")

Total de filas obtenidas: 25857


#### ***Imprimir una muestra de los datos obtenidos***

In [21]:
for i in range(min(10, len(emergencias))):
    print(emergencias[i])

{'fecha': '2019-01-01T00:00:00.000', 'departamento': 'ANTIOQUIA', 'municipio': 'SAN JERONIMO', 'evento': 'INCENDIO ESTRUCTURAL', 'divipola': '5656', 'fallecidos': '0', 'heridos': '0', 'desaparecidos': '0', 'personas': '5', 'familias': '1', 'viviendas_destruidas': '1', 'viviendas_averiadas': '0', 'vias_averiadas': '0', 'puentes_vehiculares': '0', 'puentes_peatonales': '0', 'acueducto': '0', 'alcantarillado': '0', 'centros_de_salud': '0', 'centros_educativos': '0', 'centros_comunitarios': '0', 'hectareas': '0', 'otros_afectacion': 'No registra', 'subsidio_de_arriendo': '$    - 0', 'asistencia_no_alimentaria': '$    - 0', 'apoyo_alimentario': '$    - 0', 'materiales_construccion': '$    - 0', 'sacos_bigbag': '$    - 0', 'obras_de_emergencia': '$    - 0', 'carrotanques_motobombas_planta': '$    - 0', 'horas_maquina_retroexcavadora': '$    - 0', 'apoyo_aereo_terrestre': '$    - 0', 'fic_transferencias_economicas': '$    - 0', 'infraesctructura_tecnologica': '$    - 0', 'recursos_ejecutados'

#### ***Cargar el dataset mediante Pandas***

In [22]:
df = pd.DataFrame(emergencias)

#df.head()
# df["departamento"].value_counts()
df["fecha"].value_counts()
#print("N° de filas en el dataframe:", len(df))

fecha
2022-05-11T00:00:00.000    165
2022-03-05T00:00:00.000    114
2022-03-02T00:00:00.000    111
2022-05-18T00:00:00.000    111
2022-03-06T00:00:00.000    102
                          ... 
2021-09-24T00:00:00.000      1
2021-06-27T00:00:00.000      1
2019-12-04T00:00:00.000      1
2019-12-01T00:00:00.000      1
2019-12-23T00:00:00.000      1
Name: count, Length: 1459, dtype: int64

#### ***Enfocarnos en los departamentos de la región Pacífica***

In [23]:
df = df[df['departamento'].isin(['VALLE DEL CAUCA', 'CAUCA', 'NARIÑO', 'CHOCO'])]

#df.head()
df["departamento"].value_counts()

departamento
CAUCA              1814
NARIÑO             1175
VALLE DEL CAUCA     569
CHOCO               530
Name: count, dtype: int64

#### ***Liberarnos de ciertas columnas***
Debido a que es un dataframe demasiado grande, solamente escogemos ciertas columnas de nuestro interés.

In [24]:
df = df[["fecha", "departamento", "municipio", "evento", "fallecidos", "heridos", "desaparecidos", "personas", "familias", "viviendas_destruidas", "hectareas"]]
df.head(5)

,fecha,departamento,municipio,evento,fallecidos,heridos,desaparecidos,personas,familias,viviendas_destruidas,hectareas
95,2019-01-05T00:00:00.000,VALLE DEL CAUCA,JAMUNDI,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,1
124,2019-01-06T00:00:00.000,VALLE DEL CAUCA,BUGALAGRANDE,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,5
128,2019-01-08T00:00:00.000,CAUCA,PATIA,ACCIDENTE TRANSPORTE TERRESTRE,0,15,0,15,0,0,0
132,2019-01-06T00:00:00.000,VALLE DEL CAUCA,GUADALAJARA DE BUGA,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,0
134,2019-01-08T00:00:00.000,VALLE DEL CAUCA,BUENAVENTURA,INUNDACION,1,0,0,1,0,0,0


#### ***Empezar a clasificar ciertos eventos***
Escogimos determinados eventos que, con el pasar del tiempo, se van intensificando a medida que el cambio climático empeora.

In [25]:
#df["evento"].value_counts()
df["evento"].unique()

array(['INCENDIO DE COBERTURA VEGETAL', 'ACCIDENTE TRANSPORTE TERRESTRE',
       'INUNDACION', 'VENDAVAL', 'GRANIZADA', 'MOVIMIENTO EN MASA',
       'EROSION', 'COLAPSO ESTRUCTURAL', 'INCENDIO ESTRUCTURAL',
       'ACTIVACIÓN VOLCANICA', 'ACCIDENTE TRANSPORTE MARITIMO O FLUVIAL',
       'INCENDIO', 'AVENIDA TORRENCIAL', 'TORMENTA ELECTRICA',
       'EXPLOSION', 'ACCIDENTE TECNOLOGICO', 'SEQUIA', 'SISMO',
       'ACCIDENTE MINERO', 'ACCIDENTE TRANSPORTE AEREO', 'TEMPORAL',
       'CRECIENTE SUBITA', 'INUNDACIÓN', 'HELADA', 'DERRAME', 'CONATO',
       'COLAPSO', 'OTROS', 'EROSION COSTERA', 'LLUVIAS', 'INMERSION',
       'ACCIDENTE TRANSPORTE MARÍTIMO O FLUVIAL', 'CASO FORTUITO'],
      dtype=object)

In [26]:
df = df[df["evento"].isin(["INUNDACION", "VENDAVAL", "GRANIZADA", "TORMENTA ELECTRICA", "SEQUIA", "CRECIENTE SUBITA", "INUNDACIÓN", "HELADA", "LLUVIAS"])]

df

,fecha,departamento,municipio,evento,fallecidos,heridos,desaparecidos,personas,familias,viviendas_destruidas,hectareas
134,2019-01-08T00:00:00.000,VALLE DEL CAUCA,BUENAVENTURA,INUNDACION,1,0,0,1,0,0,0
150,2019-01-08T00:00:00.000,VALLE DEL CAUCA,BUENAVENTURA,INUNDACION,1,0,0,1,0,0,0
182,2019-01-11T00:00:00.000,NARIÑO,BARBACOAS,VENDAVAL,0,0,0,118,31,0,0
186,2019-01-12T00:00:00.000,NARIÑO,GUACHUCAL,GRANIZADA,0,0,0,0,0,0,1
427,2019-01-29T00:00:00.000,NARIÑO,SANTA BARBARA,INUNDACION,0,0,0,1151,379,35,0
...,...,...,...,...,...,...,...,...,...,...,...
25634,2022-12-23T00:00:00.000,CAUCA,POPAYAN,INUNDACION,0,0,0,0,0,0,0
25640,2022-12-26T00:00:00.000,CAUCA,ROSAS,INUNDACION,0,0,0,0,0,0,0
25654,2022-12-26T00:00:00.000,CAUCA,PADILLA,INUNDACION,0,0,0,0,0,0,0
25851,2022-11-25T00:00:00.000,VALLE DEL CAUCA,DEPARTAMENTO,LLUVIAS,0,0,0,0,0,0,0


#### ***Transformación de los datos correspondiente a la hora***

In [27]:
df['fecha'] = pd.to_datetime(df['fecha'])

df['año'] = df['fecha'].dt.year

# Reorganizar las columnas
cols = ['año'] + [col for col in df.columns if col not in ['año']]
df = df[cols]

df

,año,fecha,departamento,municipio,evento,fallecidos,heridos,desaparecidos,personas,familias,viviendas_destruidas,hectareas
134,2019,2019-01-08,VALLE DEL CAUCA,BUENAVENTURA,INUNDACION,1,0,0,1,0,0,0
150,2019,2019-01-08,VALLE DEL CAUCA,BUENAVENTURA,INUNDACION,1,0,0,1,0,0,0
182,2019,2019-01-11,NARIÑO,BARBACOAS,VENDAVAL,0,0,0,118,31,0,0
186,2019,2019-01-12,NARIÑO,GUACHUCAL,GRANIZADA,0,0,0,0,0,0,1
427,2019,2019-01-29,NARIÑO,SANTA BARBARA,INUNDACION,0,0,0,1151,379,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25634,2022,2022-12-23,CAUCA,POPAYAN,INUNDACION,0,0,0,0,0,0,0
25640,2022,2022-12-26,CAUCA,ROSAS,INUNDACION,0,0,0,0,0,0,0
25654,2022,2022-12-26,CAUCA,PADILLA,INUNDACION,0,0,0,0,0,0,0
25851,2022,2022-11-25,VALLE DEL CAUCA,DEPARTAMENTO,LLUVIAS,0,0,0,0,0,0,0


#### ***Guardar la nueva versión del dataset***

In [28]:
df.to_csv("../EmergenciasCC_RegionPacifico.csv", index=False)